In [7]:
from dotenv import load_dotenv
import os
import json
import pandas as pd
import base64
from requests import post, get
import csv

load_dotenv()

client_id = os.getenv("CID")
client_secret = os.getenv("SECRET")

In [8]:
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}

    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]

    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [25]:
import random

def getRandomSearch():
    # Una lista de todos los caracteres que se pueden elegir.
    characters = 'abcdefghijklmnopqrstuvwxyz1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZñÑ ¿?:;-,><&/()=!¡|@#$"'
    
    # Obtiene un carácter aleatorio de la cadena de caracteres.
    randomCharacter = random.choice(characters)
    randomSearch = ''
    
    # Coloca el carácter comodín al principio, o al principio y al final, de forma aleatoria.
    if random.randint(0, 1) == 0:
        randomSearch = randomCharacter + '%'
    else:
        randomSearch = '%' + randomCharacter + '%'
    
    return randomSearch

def search_banch_of_tracks(randomSearch, randomOffset, limit=50):
    token = get_token()
    auth_header = get_auth_header(token)
    url = "https://api.spotify.com/v1/search"
    params = {
        "q": randomSearch,
        "type": "track",
        "limit": limit,
        "offset": randomOffset,
    }
    response = get(url, headers=auth_header, params=params)
    json_response = json.loads(response.content)
    return json_response

def write_csv(json_response):
    try:
        new_tracks_df = pd.DataFrame(json_response["tracks"]["items"])
        with open("db_extract.csv", "a") as f:
            new_tracks_df.to_csv(f)
    except:
        pass
        
def depurar_csv():
    df = pd.read_csv("db_extract.csv")
    df.drop_duplicates(subset=["id"], inplace=True)
    df.to_csv("db_extract.csv", index=False)

In [22]:
def get_several_tracks(iterations=1000):
    for i in range(0, iterations):
        randomSearch = getRandomSearch()
        randomOffset = random.randint(0, 1000)
        print(f'Iteration {i} - Search: {randomSearch} - Offset: {randomOffset}')
        json_response = search_banch_of_tracks(randomSearch, randomOffset)
        write_csv(json_response)
    depurar_csv()

In [28]:
get_several_tracks(iterations=10000)

Iteration 0 - Search: %V% - Offset: 727
Iteration 1 - Search: 6% - Offset: 634
Iteration 2 - Search: %=% - Offset: 498
Iteration 3 - Search: %Y% - Offset: 611
Iteration 4 - Search: %y% - Offset: 431
Iteration 5 - Search: %=% - Offset: 3
Iteration 6 - Search: %3% - Offset: 767
Iteration 7 - Search: %D% - Offset: 4
Iteration 8 - Search: %>% - Offset: 391
Iteration 9 - Search: p% - Offset: 294
Iteration 10 - Search: %L% - Offset: 440
Iteration 11 - Search: ;% - Offset: 410
Iteration 12 - Search: %m% - Offset: 699
Iteration 13 - Search: A% - Offset: 19
Iteration 14 - Search: w% - Offset: 766
Iteration 15 - Search: e% - Offset: 538
Iteration 16 - Search: %ñ% - Offset: 789
Iteration 17 - Search: ;% - Offset: 935
Iteration 18 - Search: 4% - Offset: 565
Iteration 19 - Search: %O% - Offset: 325
Iteration 20 - Search: %X% - Offset: 736
Iteration 21 - Search: &% - Offset: 30
Iteration 22 - Search: %>% - Offset: 236
Iteration 23 - Search: %e% - Offset: 596
Iteration 24 - Search: k% - Offset: 106
I

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [29]:
depurar_csv()
df = pd.read_csv("db_extract.csv")
df.shape

(21482, 18)